# Forest Fires UCI Dataset

In [ ]:
# set python to pick up local directory
from pathlib import Path
import sys
import os
path_root = Path(os.getcwd()).parents[0]
sys.path.append(str(path_root))
sys.path.append(str(Path(os.getcwd()).parents[1]))

%reload_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter('always', category=FutureWarning)

In [ ]:
from jax import config
config.update("jax_enable_x64", True)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
import jax
import jax.numpy as jnp
from jax import vmap, jit
from tensorflow_probability.substrates.jax import distributions as tfd
import optax
import equinox as eqx
from jax.lib import xla_bridge

In [ ]:
from steinRF import GP, LowRankGP, MixGP
from steinRF.gp.kernels import RFF
from steinRF.gp.transforms import Transform, ARD

from steinRF.stein.srfr import srfr
# from steinRF.mar_srfr import mar_srfr
from steinRF.utils import gp_cross_val, metric_model, run_hyperopt, mse, mae
from steinRF.baselines import build_svgp, build_train_svgp, svgp_predict, svgp_cross_val
from steinRF.baselines import orthogonal_gaussian
from steinRF.gp.models import *

In [ ]:
jax.clear_caches()
print(f"device: {xla_bridge.get_backend().platform}")

device: gpu


## Data Preparation

In [ ]:
key = jax.random.PRNGKey(0)

In [ ]:
wine = pd.read_csv("wine.csv")
X, y = jnp.array(wine.iloc[:, :-1]), jnp.array(wine["quality"])
y = y.reshape(-1)
X_scaler = StandardScaler().fit(X)
X = jnp.array(X_scaler.transform(X))
y_scaler = StandardScaler().fit(y.reshape(-1, 1))
y = jnp.array(y_scaler.transform(y.reshape(-1, 1)).reshape(-1))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=int(key[0]))
X_train, X_val, y_train, y_val = jnp.array(X_train), jnp.array(X_val), jnp.array(y_train), jnp.array(y_val)